In [49]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import hamming

In [9]:
ratings=pd.read_csv('C:/Users/Kandarp/Desktop/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [10]:
movies=pd.read_csv('C:/Users/Kandarp/Desktop/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [21]:
def moviemeta(movieid):
    title=movies.at[movieid,'title']
    return title
    

In [22]:
print(moviemeta(5))

Heat (1995)


In [30]:
def favmovie(userId,N):
    userRatings= ratings[ratings['userId']==userId]
    sortedRatings= pd.DataFrame.sort_values(userRatings,['rating'],ascending=[0])[:N]
    sortedRatings['title']= sortedRatings['movieId'].apply(moviemeta)
    return sortedRatings

In [31]:
favmovie(1,5)

,userId,movieId,rating,timestamp,title
170,1,8507,5.0,1094786027,"Barkleys of Broadway, The (1949)"
131,1,4993,5.0,1112484682,Honky Tonk Freeway (1981)
142,1,5952,5.0,1112484619,"Harder They Come, The (1973)"
158,1,7153,5.0,1112484633,"Dreamers, The (2003)"
30,1,1196,4.5,1112484742,Full Metal Jacket (1987)


In [41]:
ratings.shape
movies.shape

(27278, 3)

In [35]:
uniMovieId= ratings.movieId.value_counts()
uniMovieId.shape

In [38]:
uniUserId= ratings.userId.value_counts()
uniUserId.shape

(7120,)

In [43]:
matrix=pd.pivot_table(ratings,values='rating',index=['userId'],columns=['movieId'])

In [44]:
matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,129350,129354,129428,129707,130052,130073,130219,130462,130490,130642
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
def distance(user1,user2):
    user1Ratings = matrix.transpose()[user1]
    user2Ratings = matrix.transpose()[user2]
    distance=hamming(user1Ratings,user2Ratings)
        #distance=np.NaN
    return distance

In [66]:
distance(1,3)

0.9995722230144019

In [75]:
def nearestNeighbours(userId,K=10):
    allUsers=pd.DataFrame(matrix.index)
    allUsers=allUsers[allUsers.userId!=userId]
    allUsers['distance']=allUsers['userId'].apply(lambda x: distance(userId,x))
    KnearestUsers=allUsers.sort_values(['distance'],ascending=True)['userId'][:K]
    return KnearestUsers

In [87]:
#matrix.head()

In [95]:
def topN(user,N=3):
    k=nearestNeighbours(user)
    NNRatings=matrix[matrix.index.isin(k)]
    avgRating= NNRatings.apply(np.nanmean).dropna()
    movieSeen=matrix.transpose()[user].dropna().index
    avgRating=avgRating[~avgRating.index.isin(movieSeen)]
    topMovies=avgRating.sort_values(ascending=False).index[:N]
    return pd.Series(topMovies).apply(moviemeta)

In [93]:
favmovie(1,5)

,userId,movieId,rating,timestamp,title
170,1,8507,5.0,1094786027,"Barkleys of Broadway, The (1949)"
131,1,4993,5.0,1112484682,Honky Tonk Freeway (1981)
142,1,5952,5.0,1112484619,"Harder They Come, The (1973)"
158,1,7153,5.0,1112484633,"Dreamers, The (2003)"
30,1,1196,4.5,1112484742,Full Metal Jacket (1987)


In [96]:
topN(1,5)

C:\Users\Kandarp\Anaconda3\lib\site-packages\pandas\core\apply.py:242: RuntimeWarning: Mean of empty slice
  labels=labels)


0               Tea with Mussolini (1999)
1                      North Shore (1987)
2                        Beautiful (2000)
3                    Scotland, Pa. (2001)
4    Ichi the Killer (Koroshiya 1) (2001)
Name: movieId, dtype: object